# dbApps08c Task: Database Constraints

In this task, you will explore SQL constraints that enforce data integrity:
- **NOT NULL**: Ensures a column must always have a value
- **UNIQUE**: Ensures all values in a column are different
- **DEFAULT**: Provides a default value if none is specified
- **CHECK**: Validates that values meet a condition
- **FOREIGN KEY**: Links data between tables

You will create tables with these constraints, attempt to violate them (to see errors), and understand why they matter.

**Database**: `task08c.db`

In [ ]:
# Import required libraries
import pandas as pd
import sqlite3

# Create (or connect to) the database
conn = sqlite3.connect("task08c.db")
print("Database connected: task08c.db")

---

## TASK 1: NOT NULL Constraint

Create a `customers` table where firstName and lastName MUST have values.
Then insert a valid record and verify it.

In [ ]:
# Create the customers table with NOT NULL constraints on firstName and lastName
createCustomersSQL = """
CREATE TABLE IF NOT EXISTS customers (
    customerId INTEGER PRIMARY KEY AUTOINCREMENT,
    firstName TEXT NOT NULL,
    lastName TEXT NOT NULL,
    email TEXT
)
"""

conn.execute(createCustomersSQL)
conn.commit()
print("✓ customers table created with NOT NULL on firstName and lastName")

In [ ]:
# Insert a valid record with both firstName and lastName specified
insertValidCustomerSQL = """
INSERT INTO customers (firstName, lastName, email)
VALUES ('John', 'Smith', 'john.smith@email.com')
"""

conn.execute(insertValidCustomerSQL)
conn.commit()
print("✓ Valid customer record inserted")

# Verify the insertion
verifyData = pd.read_sql("SELECT * FROM customers", conn)
print("\nCustomers table:")
print(verifyData)

### Task 1 Reflection

What happens if you try to insert a record with NULL for firstName?

**Answer**: If you attempt to execute:
```sql
INSERT INTO customers (firstName, lastName, email)
VALUES (NULL, 'Johnson', 'johnson@email.com')
```

SQLite will throw a **NOT NULL constraint violation error** similar to:
```
sqlite3.IntegrityError: NOT NULL constraint failed: customers.firstName
```

The INSERT will fail, and no row will be added to the table. This prevents incomplete or invalid customer records from being stored.

---

## TASK 2: UNIQUE Constraint

Add a UNIQUE constraint to the email column so no two customers can have the same email.
Insert multiple customers with different emails, then describe what happens with a duplicate.

In [ ]:
# Drop the old table and recreate it with UNIQUE on email
conn.execute("DROP TABLE IF EXISTS customers")
conn.commit()

# Create customers table with UNIQUE constraint on email
createCustomersWithUniqueSQL = """
CREATE TABLE customers (
    customerId INTEGER PRIMARY KEY AUTOINCREMENT,
    firstName TEXT NOT NULL,
    lastName TEXT NOT NULL,
    email TEXT UNIQUE
)
"""

conn.execute(createCustomersWithUniqueSQL)
conn.commit()
print("✓ customers table recreated with UNIQUE constraint on email")

In [ ]:
# Insert two customers with different emails
insertCustomer1SQL = """
INSERT INTO customers (firstName, lastName, email)
VALUES ('Alice', 'Brown', 'alice.brown@email.com')
"""

insertCustomer2SQL = """
INSERT INTO customers (firstName, lastName, email)
VALUES ('Bob', 'Green', 'bob.green@email.com')
"""

conn.execute(insertCustomer1SQL)
conn.execute(insertCustomer2SQL)
conn.commit()
print("✓ Two customers with different emails inserted")

# Verify
verifyData = pd.read_sql("SELECT * FROM customers", conn)
print("\nCustomers table:")
print(verifyData)

### Task 2 Reflection

What happens if you try to insert a customer with a duplicate email?

**Answer**: If you attempt to execute:
```sql
INSERT INTO customers (firstName, lastName, email)
VALUES ('Charlie', 'White', 'alice.brown@email.com')
```

SQLite will throw a **UNIQUE constraint violation error** similar to:
```
sqlite3.IntegrityError: UNIQUE constraint failed: customers.email
```

The INSERT will fail because 'alice.brown@email.com' already exists in the table. This ensures data integrity by preventing duplicate email addresses in the customer database.

---

## TASK 3: DEFAULT Constraint

Create a `products` table with a DEFAULT value for the category column.
Insert a product WITHOUT specifying the category and verify the default was applied.

In [ ]:
# Create products table with DEFAULT value for category
createProductsSQL = """
CREATE TABLE products (
    productId INTEGER PRIMARY KEY AUTOINCREMENT,
    productName TEXT NOT NULL,
    category TEXT DEFAULT 'General',
    price REAL NOT NULL
)
"""

conn.execute(createProductsSQL)
conn.commit()
print("✓ products table created with DEFAULT 'General' for category")

In [ ]:
# Insert a product WITHOUT specifying the category
# The DEFAULT value should be automatically applied
insertProductNoDefaultSQL = """
INSERT INTO products (productName, price)
VALUES ('Mystery Item', 19.99)
"""

conn.execute(insertProductNoDefaultSQL)
conn.commit()
print("✓ Product inserted WITHOUT specifying category")

# Also insert a product WITH a specific category for comparison
insertProductWithCategorySQL = """
INSERT INTO products (productName, category, price)
VALUES ('Laptop', 'Electronics', 999.99)
"""

conn.execute(insertProductWithCategorySQL)
conn.commit()
print("✓ Product inserted WITH a specified category")

# Verify that the default was applied
verifyData = pd.read_sql("SELECT * FROM products", conn)
print("\nProducts table:")
print(verifyData)

### Task 3 Reflection

The DEFAULT constraint automatically assigns a value to a column if the user does not provide one during INSERT.

In the example above:
- 'Mystery Item' was inserted without specifying a category, so it automatically received the DEFAULT value **'General'**
- 'Laptop' was inserted with an explicit category of **'Electronics'**, so the DEFAULT was not used

This is useful for setting sensible defaults and reducing the need for users to specify every column value.

---

## TASK 4: CHECK Constraint

Add a CHECK constraint to the products table to ensure price is always positive.
Describe what happens if you try to insert a negative price.

In [ ]:
# Drop the old products table and recreate it with CHECK constraint on price
conn.execute("DROP TABLE IF EXISTS products")
conn.commit()

# Create products table with CHECK constraint
createProductsWithCheckSQL = """
CREATE TABLE products (
    productId INTEGER PRIMARY KEY AUTOINCREMENT,
    productName TEXT NOT NULL,
    category TEXT DEFAULT 'General',
    price REAL NOT NULL CHECK(price > 0)
)
"""

conn.execute(createProductsWithCheckSQL)
conn.commit()
print("✓ products table recreated with CHECK(price > 0) constraint")

In [ ]:
# Insert a valid product with positive price
insertValidProductSQL = """
INSERT INTO products (productName, category, price)
VALUES ('Headphones', 'Electronics', 79.99)
"""

conn.execute(insertValidProductSQL)
conn.commit()
print("✓ Valid product with positive price inserted")

# Verify
verifyData = pd.read_sql("SELECT * FROM products", conn)
print("\nProducts table:")
print(verifyData)

### Task 4 Reflection

What happens if you try to insert a product with a negative price?

**Answer**: If you attempt to execute:
```sql
INSERT INTO products (productName, price)
VALUES ('Free Item', -9.99)
```

SQLite will throw a **CHECK constraint violation error** similar to:
```
sqlite3.IntegrityError: CHECK constraint failed: products
```

The INSERT will fail because the CHECK constraint `price > 0` is violated. This ensures that all product prices are positive, preventing nonsensical data like negative prices from entering the database.

---

## TASK 5: FOREIGN KEY Constraint (Part 1)

Create an `orders` table with a FOREIGN KEY that references the customers table.
Insert a valid order (with a customerId that exists in the customers table).

In [ ]:
# First, recreate the customers table (clean slate)
conn.execute("DROP TABLE IF EXISTS customers")
conn.commit()

createCustomersSQL = """
CREATE TABLE customers (
    customerId INTEGER PRIMARY KEY AUTOINCREMENT,
    firstName TEXT NOT NULL,
    lastName TEXT NOT NULL,
    email TEXT UNIQUE
)
"""

conn.execute(createCustomersSQL)
conn.commit()
print("✓ customers table created")

In [ ]:
# Insert a customer to reference in the orders table
insertCustomerSQL = """
INSERT INTO customers (firstName, lastName, email)
VALUES ('Sarah', 'Johnson', 'sarah.j@email.com')
"""

conn.execute(insertCustomerSQL)
conn.commit()
print("✓ Customer inserted (customerId will be 1)")

# Verify customer was created
verifyCustomers = pd.read_sql("SELECT * FROM customers", conn)
print("\nCustomers table:")
print(verifyCustomers)

In [ ]:
# Create orders table with FOREIGN KEY constraint referencing customers table
createOrdersSQL = """
CREATE TABLE orders (
    orderId INTEGER PRIMARY KEY AUTOINCREMENT,
    customerId INTEGER NOT NULL,
    orderDate TEXT NOT NULL,
    totalAmount REAL NOT NULL,
    FOREIGN KEY (customerId) REFERENCES customers(customerId)
)
"""

conn.execute(createOrdersSQL)
conn.commit()
print("✓ orders table created with FOREIGN KEY referencing customers(customerId)")

In [ ]:
# Insert a valid order with customerId that exists in the customers table
insertValidOrderSQL = """
INSERT INTO orders (customerId, orderDate, totalAmount)
VALUES (1, '2025-02-10', 149.99)
"""

conn.execute(insertValidOrderSQL)
conn.commit()
print("✓ Valid order inserted (customerId 1 exists in customers table)")

# Verify
verifyOrders = pd.read_sql("SELECT * FROM orders", conn)
print("\nOrders table:")
print(verifyOrders)

### Task 5 Reflection (Part 1)

The FOREIGN KEY constraint ensures **referential integrity** between two tables.

In this example:
- The `orders` table's `customerId` column is a FOREIGN KEY
- It references the `customers` table's `customerId` column
- This means every order MUST be associated with a valid customer

We successfully inserted an order with customerId = 1, which exists in the customers table.

---

## TASK 5 CONTINUED: FOREIGN KEY Constraint (Part 2)

Describe what happens if you try to insert an order with a customerId that does NOT exist.

### Task 5 Reflection (Part 2)

What happens if you try to insert an order with a customerId that doesn't exist in the customers table?

**Answer**: If you attempt to execute:
```sql
INSERT INTO orders (customerId, orderDate, totalAmount)
VALUES (999, '2025-02-10', 50.00)
```

SQLite will throw a **FOREIGN KEY constraint violation error** similar to:
```
sqlite3.IntegrityError: FOREIGN KEY constraint failed
```

The INSERT will fail because customerId 999 does not exist in the customers table. This prevents "orphan" orders from being created—orders that reference non-existent customers. Foreign key constraints maintain data consistency across related tables.

---

## TASK 6: Combining ALL Constraints

Create a single table that combines NOT NULL, UNIQUE, DEFAULT, CHECK, and FOREIGN KEY constraints.
This demonstrates how multiple constraints can work together on one table.

In [ ]:
# Create an invoices table that uses all types of constraints
# Note: We already have customers and products tables

createInvoicesSQL = """
CREATE TABLE invoices (
    invoiceId INTEGER PRIMARY KEY AUTOINCREMENT,
    customerId INTEGER NOT NULL,
    productId INTEGER NOT NULL,
    quantity INTEGER NOT NULL CHECK(quantity > 0),
    invoiceDate TEXT NOT NULL,
    status TEXT DEFAULT 'Pending',
    notes TEXT UNIQUE,
    FOREIGN KEY (customerId) REFERENCES customers(customerId),
    FOREIGN KEY (productId) REFERENCES products(productId)
)
"""

conn.execute(createInvoicesSQL)
conn.commit()
print("✓ invoices table created with multiple constraints:")
print("  - NOT NULL: customerId, productId, quantity, invoiceDate")
print("  - CHECK: quantity > 0")
print("  - DEFAULT: status = 'Pending'")
print("  - UNIQUE: notes")
print("  - FOREIGN KEY: customerId references customers")
print("  - FOREIGN KEY: productId references products")

---

## TASK 7: Insert Valid Records into Fully Constrained Table

Insert 3 valid records into the invoices table and verify they were inserted.

In [ ]:
# Verify the customers table has at least one customer (we already inserted one)
customersCheck = pd.read_sql("SELECT * FROM customers", conn)
print("Customers in database:")
print(customersCheck)

In [ ]:
# Verify the products table has at least one product (we already inserted one)
productsCheck = pd.read_sql("SELECT * FROM products", conn)
print("\nProducts in database:")
print(productsCheck)

In [ ]:
# Insert 3 valid invoices
# Invoice 1: Valid with all values
insertInvoice1SQL = """
INSERT INTO invoices (customerId, productId, quantity, invoiceDate, status, notes)
VALUES (1, 1, 2, '2025-02-10', 'Paid', 'Bulk order - winter sale')
"""

# Invoice 2: Without specifying status (should use DEFAULT 'Pending')
insertInvoice2SQL = """
INSERT INTO invoices (customerId, productId, quantity, invoiceDate)
VALUES (1, 1, 1, '2025-02-11')
"""

# Invoice 3: Another valid invoice
insertInvoice3SQL = """
INSERT INTO invoices (customerId, productId, quantity, invoiceDate, notes)
VALUES (1, 1, 5, '2025-02-09', 'High priority shipment')
"""

conn.execute(insertInvoice1SQL)
conn.execute(insertInvoice2SQL)
conn.execute(insertInvoice3SQL)
conn.commit()

print("✓ Three valid invoices inserted")

# Verify all invoices
verifyInvoices = pd.read_sql("SELECT * FROM invoices", conn)
print("\nInvoices table:")
print(verifyInvoices)

---

## TASK 8: Reflection - Why Are Constraints Important?

### What problems do constraints prevent?

**1. NOT NULL prevents incomplete data:**
   - Without NOT NULL, critical fields like firstName or customerId could be missing
   - This would make records unusable or confusing
   - Example: An order with no customerId cannot be fulfilled

**2. UNIQUE prevents duplicates:**
   - Email addresses must be unique to identify customers correctly
   - Without UNIQUE, the same email could appear multiple times, causing confusion
   - Example: Two customers with the same email might receive the wrong orders

**3. CHECK prevents invalid values:**
   - A price cannot be negative (it doesn't make business sense)
   - A quantity must be positive
   - Without CHECK, nonsensical data could be stored
   - Example: A product with price = -$100 would corrupt inventory records

**4. DEFAULT reduces data entry burden:**
   - Common values like status = 'Pending' don't need to be typed every time
   - Reduces user errors and makes data consistent
   - Example: New orders automatically start in 'Pending' status

**5. FOREIGN KEY ensures referential integrity:**
   - Orders must belong to real customers
   - Prevents "orphan" records with broken links
   - Without FOREIGN KEY, you could have orders referencing customers that don't exist
   - Example: An invoice linking to productId 999 when only 5 products exist is nonsensical

### Summary
**Constraints are guardrails that keep your database clean, consistent, and reliable.**
They prevent data quality issues, reduce bugs, and save time by catching errors at the source rather than in application code.
A well-constrained database is easier to maintain and less likely to cause business logic errors.